In [0]:
import os

from bs4 import BeautifulSoup
import requests

import pandas as pd

if os.path.exists('infomotion.csv'):
    os.remove('infomotion.csv')
  
def scan_csv_files(folder_path):  
    for filename in os.listdir(folder_path):  
        file_path = os.path.join(folder_path, filename)  
        if os.path.isdir(file_path):  
            scan_csv_files(file_path) # rekursiver Aufruf der Funktion, wenn der Eintrag ein Ordner ist  
        elif filename.endswith('.html'):            
            print("HTML-Datei gefunden:", file_path)
            
            # Assign URL
            url = file_path
            #print(url)
            
            with open(file_path , 'rb') as html:
                soup = BeautifulSoup(html)
                                    
            title = soup.find('title')
            display(title.string)
            
            # In hmtl-Title kommt oft noch " | INFOMOTION" am Ende, das nehmen wir raus
            title_string = title.string.replace(" | INFOMOTION", "")
            
            # Die URL ersetzen wir durch die richtige URL
            url_string = url.replace("D:\infomotion_website\infomotion\www.infomotion.de", "https://www.infomotion.de")
            # Backslash \ durch Slash / ersetzen
            url_string = url_string.replace(os.sep, "/")
            # Dateinamenendung ".html" entfernen
            url_string = url_string.replace(".html", "")
            print(url_string)
            
            texts = soup.find_all('p')            
            df = pd.DataFrame(texts, columns=['Text'])
            df['Kategorie'] = title_string
            df['Source'] = url_string
            
            try:
                for index, row in df.iterrows():
                    r = row['Text'].get_text().split()
                    if len(r) == 1:
                        df.drop(index, inplace=True)
            except:
                pass
                       
            #in infomotion.csv Text + Kategorie + Source schreiben im Append-Modus
            df.to_csv('infomotion.csv', mode='a', header = True, index = False, sep = '|')

            
folder_path = "D:\infomotion_website\infomotion" # Ersetze "Pfad zum Ordner" durch den Pfad zu deinem Ordner  
scan_csv_files(folder_path)  
